In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [35]:
# Vancouver latitude and longitude: 49.2827° N, 123.1207° W
cherry_sub2 <- cherry_sub %>%
    filter(123 < lat & lat < 125)
sort(unique(cherry_sub2$city)) # Iriomotejima, Ishigakijima, Yonagunijima
dim(cherry_sub2)
head(cherry_sub2)
unique(cherry_sub2$alt)


[1] "Iriomotejima" "Ishigakijima" "Yonagunijima"

[1] 95  9

,country,city,location,lat,long,alt,year,bloom_date,bloom_doy
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<int>
1,Japan,Yonagunijima,Japan/Yonagunijima,24.46667,123.0106,30,1992,1992-02-10,41
2,Japan,Yonagunijima,Japan/Yonagunijima,24.46667,123.0106,30,1993,1993-02-17,48
3,Japan,Yonagunijima,Japan/Yonagunijima,24.46667,123.0106,30,1994,1994-02-07,38
4,Japan,Yonagunijima,Japan/Yonagunijima,24.46667,123.0106,30,1995,1995-03-02,61
5,Japan,Yonagunijima,Japan/Yonagunijima,24.46667,123.0106,30,1996,1996-02-17,48
6,Japan,Yonagunijima,Japan/Yonagunijima,24.46667,123.0106,30,1997,1997-02-12,43


[1] 30.0  9.9  5.7

In [26]:
cherry_sub <- read.csv("/home/joosungm/projects/def-lelliott/joosungm/projects/peak-bloom-prediction/A_outputs/A11_cherry_sub.csv") %>%
    filter(city == "Liestal")
dim(cherry_sub)
head(cherry_sub)

[1] 14080     9

,country,city,location,lat,long,alt,year,bloom_date,bloom_doy
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<int>
1,Switzerland,Enges,Switzerland/Enges,47.05644,7.012192,820,1951,1951-05-05,125
2,Switzerland,Fiesch,Switzerland/Fiesch,46.40026,8.127792,1100,1951,1951-04-25,115
3,Switzerland,Liddes,Switzerland/Liddes,45.98827,7.180528,1350,1951,1951-05-22,142
4,Switzerland,Les Ponts-de-Martel,Switzerland/Les Ponts-de-Martel,47.00286,6.728592,1120,1951,1951-05-29,149
5,Switzerland,Orvin,Switzerland/Orvin,47.15775,7.214464,700,1951,1951-04-27,117
6,Switzerland,Posieux,Switzerland/Posieux,46.76167,7.098294,680,1951,1951-04-26,116


In [16]:
target_years <- sort(unique(cherry_sub$year))
target_years

[1] 1952 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999
[16] 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014
[31] 2015 2016 2017 2018 2019 2020 2021 2022

In [18]:
temp_data <- read.csv("/home/joosungm/projects/def-lelliott/joosungm/projects/peak-bloom-prediction/A_outputs/A21_liestal_temperature.csv") %>%
    filter(year %in% target_years)
dim(temp_data)
head(temp_data)

[1] 12906     9

,X,id,date,tmax,tmin,prcp,year,month,day
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
1,11811,GME00127786,1986-01-01,-0.5,-5.5,2.6,1986,1,1
2,11812,GME00127786,1986-01-02,6.0,-1.0,0.7,1986,1,2
3,11813,GME00127786,1986-01-03,9.6,2.3,2.3,1986,1,3
4,11814,GME00127786,1986-01-04,5.4,-0.8,0.0,1986,1,4
5,11815,GME00127786,1986-01-05,2.6,-3.8,0.3,1986,1,5
6,11816,GME00127786,1986-01-06,3.1,0.8,1.0,1986,1,6


In [19]:
# Chill-day model (Cesaraccio et al., 2004)
# - Consists of 6 cases: (Tc: threshold temp, Tn = daily minimum, Tx = daily maximum, Tm = daily mean, Cd = chill days, Ca = anti-chill days)

chill_days <- function(r, Tc = 7) {
    
    Tmin <- as.numeric(r[["tmin"]])
    Tmax <- as.numeric(r[["tmax"]])
    Tmean <- (Tmin + Tmax)/2

    if (0 <= Tc & Tc <= Tmin & Tmin <= Tmax) {
        
        Cd <- 0
        Ca <- Tmean - Tc

    } else if (0 <= Tmin & Tmin <= Tc & Tc <= Tmax) {

        Cd <- -1 * ((Tmean - Tmin) - ((Tmax - Tc)/2))
        Ca <- (Tmax - Tc)/2

    } else if (0 <= Tmin & Tmin <= Tmax & Tmax <= Tc) {

        Cd <- -1 * (Tmean - Tmin)
        Ca <- 0
    
    } else if (Tmin <= 0 & 0 <= Tmax & Tmax <= Tc) {

        Cd <- -1 * (Tmax / (Tmax - Tmin)) * (Tmax/2)
        Ca <- 0

    } else if (Tmin <= 0 & 0 <= Tc & Tc <= Tmax) {

        Cd <- -1 * ((Tmax / (Tmax - Tmin)) * (Tmax/2) - ((Tmax - Tc)/2))
        Ca <- (Tmax - Tc) / 2

    } else if (Tmax < 0) {

        Cd <- 0
        Ca <- 0

    } else {
        
        Cd <- 0
        Ca <- 0
    }

    return(c(Cd, Ca))
}

temp_data$daily_Cd <- apply(temp_data, MARGIN = 1, FUN = function(x) {chill_days(r = x)[1]})
temp_data$daily_Ca <- apply(temp_data, MARGIN = 1, FUN = function(x) {chill_days(r = x)[2]})

head(temp_data)

,X,id,date,tmax,tmin,prcp,year,month,day,daily_Cd,daily_Ca
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>
1,11811,GME00127786,1986-01-01,-0.5,-5.5,2.6,1986,1,1,0.000000,0.0
2,11812,GME00127786,1986-01-02,6.0,-1.0,0.7,1986,1,2,-2.571429,0.0
3,11813,GME00127786,1986-01-03,9.6,2.3,2.3,1986,1,3,-2.350000,1.3
4,11814,GME00127786,1986-01-04,5.4,-0.8,0.0,1986,1,4,-2.351613,0.0
5,11815,GME00127786,1986-01-05,2.6,-3.8,0.3,1986,1,5,-0.528125,0.0
6,11816,GME00127786,1986-01-06,3.1,0.8,1.0,1986,1,6,-1.150000,0.0


In [22]:
# Obtain cumsum of Cd and Ca
# - Cd: start from (yr-1)-10-01
# - Ca: start from yr-01-31
# Merge with the bloom_date data.
# Plot histogram of the Ca_cumsum of the bloom_date.

temp_data_list <- list()

for (yr in target_years) {

    sub_df <- temp_data[paste0(as.character(yr-1), "-09-30") < temp_data$date & temp_data$date < paste0(as.character(yr), "-05-01"), ]
    sub_df$Cd_cumsum <- cumsum(sub_df$daily_Cd)
    sub_df2 <- sub_df[sub_df$date > as.Date(paste0(as.character(yr), "-01-01")), ]
    sub_df2$Ca_cumsum <- cumsum(sub_df2$daily_Ca)
    combined_df <- sub_df %>% merge(y = sub_df2[, c("date", "Ca_cumsum")], by = "date", all.x = TRUE)
    temp_data_list[[yr]] <- combined_df
    
}
# print("done")

temp_complete <- temp_data_list %>% bind_rows() %>%
    filter(!is.na(Ca_cumsum))
# dim(temp_complete)
# head(temp_complete)

cherry_bloom_date <- read.csv("/home/joosungm/projects/def-lelliott/joosungm/projects/peak-bloom-prediction/A_outputs/A11_cherry_sub.csv") %>%
    filter(city == "Liestal") %>% dplyr::select(bloom_date, bloom_doy)

temp_complete2 <- temp_complete %>% merge(y = cherry_bloom_date, by.x = "date", by.y = "bloom_date", all.x = TRUE) %>%
    mutate(is_bloom = ifelse(!is.na(bloom_doy), 1, 0)) %>%
    filter(month > 2)

dim(temp_complete2)
head(temp_complete2)
write.csv(temp_complete2, "/home/joosungm/projects/def-lelliott/joosungm/projects/peak-bloom-prediction/A_outputs/A41_gdd_liestal.csv", row.names = FALSE)

[1] 2135   15

,date,X,id,tmax,tmin,prcp,year,month,day,daily_Cd,daily_Ca,Cd_cumsum,Ca_cumsum,bloom_doy,is_bloom
,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,1986-03-01,11870,GME00127786,1.0,-4.5,0.0,1986,3,1,-0.09090909,0.00,-50.73523,17.25,NA,0
2,1986-03-02,11871,GME00127786,2.4,-7.4,0.0,1986,3,2,-0.29387755,0.00,-51.02911,17.25,NA,0
3,1986-03-03,11872,GME00127786,1.0,-5.2,2.6,1986,3,3,-0.08064516,0.00,-51.10976,17.25,NA,0
4,1986-03-04,11873,GME00127786,3.5,-6.7,0.0,1986,3,4,-0.60049020,0.00,-51.71025,17.25,NA,0
5,1986-03-05,11874,GME00127786,10.0,-0.4,0.2,1986,3,5,-3.30769231,1.50,-55.01794,18.75,NA,0
6,1986-03-06,11875,GME00127786,12.5,1.5,1.5,1986,3,6,-2.75000000,2.75,-57.76794,21.50,NA,0
